In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow import keras
import glob
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.1,
        rotation_range=20,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_path = 'code'
test_path = 'testdf'
train_generator = train_datagen.flow_from_directory(train_path,
                                       color_mode="rgb",
                                       classes=None,
                                       class_mode="categorical",
                                       batch_size=32,
                                       shuffle=True, 
                                       save_prefix='gen', 
									   target_size=(224, 224))
validation_generator = test_datagen.flow_from_directory(test_path,
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 91850 images belonging to 550 classes.
Found 4950 images belonging to 550 classes.


In [11]:
class_to_index = train_generator.class_indices
print(validation_generator.class_indices)

{'img0': 0, 'img1': 1, 'img10': 2, 'img100': 3, 'img101': 4, 'img102': 5, 'img103': 6, 'img104': 7, 'img105': 8, 'img106': 9, 'img107': 10, 'img108': 11, 'img109': 12, 'img11': 13, 'img110': 14, 'img111': 15, 'img112': 16, 'img113': 17, 'img114': 18, 'img115': 19, 'img116': 20, 'img117': 21, 'img118': 22, 'img119': 23, 'img12': 24, 'img120': 25, 'img121': 26, 'img122': 27, 'img123': 28, 'img124': 29, 'img125': 30, 'img126': 31, 'img127': 32, 'img128': 33, 'img129': 34, 'img13': 35, 'img130': 36, 'img131': 37, 'img132': 38, 'img133': 39, 'img134': 40, 'img135': 41, 'img136': 42, 'img137': 43, 'img138': 44, 'img139': 45, 'img14': 46, 'img140': 47, 'img141': 48, 'img142': 49, 'img143': 50, 'img144': 51, 'img145': 52, 'img146': 53, 'img147': 54, 'img148': 55, 'img149': 56, 'img15': 57, 'img150': 58, 'img151': 59, 'img152': 60, 'img153': 61, 'img154': 62, 'img155': 63, 'img156': 64, 'img157': 65, 'img158': 66, 'img159': 67, 'img16': 68, 'img160': 69, 'img161': 70, 'img162': 71, 'img163': 72

In [28]:
def Name(i):
    name = validation_generator.filenames[i].split("\\")[0]
    return name

In [29]:
Name(11)

'img1'

In [12]:
# example of loading the vgg16 model
from keras.applications import DenseNet169
# load model
#model = DenseNet169()
# summarize the model
base_model = DenseNet169(weights=None, include_top=False)
base_model.summary()



Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
______________________________________________________________________________________

In [13]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(550, activation='softmax')(x)

In [14]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])

In [30]:
model.fit_generator(train_generator, epochs=10,verbose=1,validation_data=validation_generator)

Epoch 1/10
1465/2871 [==============>...............] - ETA: 64:58:32 - loss: 4.5953 - accuracy: 0.0574

KeyboardInterrupt: 